In [ ]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("D:\\Github\NLP\\Artificial_Intelligence_for_NLP\\Week_08_0824_svm_naive_bayes_decision_tree\\Assignment\\cleaned_data.csv", index_col=0)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


In [6]:
df.label.value_counts()

1    78472
0     8391
Name: label, dtype: int64